In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

from matplotlib import pyplot

In [2]:
# pd.set_option('display.max_columns', None)

df = pd.read_csv('../../BPIC_Classification/processed_dataset/P_indexbased_one_15.csv')
df.head()

,Responsible_actor_0_560429,Responsible_actor_0_560431,Responsible_actor_0_560454,Responsible_actor_0_560458.0,Responsible_actor_0_560464.0,Responsible_actor_0_560519,Responsible_actor_0_560521.0,Responsible_actor_0_560530.0,Responsible_actor_0_560532.0,Responsible_actor_0_560594,...,org:resource_9_8492512.0,org:resource_9_9106499.0,org:resource_9_9264148.0,org:resource_9_10716070.0,org:resource_9_11744364.0,org:resource_9_13412010.0,org:resource_9_20987361.0,org:resource_9_22445896.0,label,time:timestamp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2010-10-06 05:28:37.000000000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2010-11-29 00:05:39.773584896
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2010-11-07 18:55:24.973684224
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2010-11-26 20:14:44.254902016
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2010-11-13 13:41:31.407407360


In [3]:
class AccuracyScore:

    def __init__(self, X_train, y_train, X_test, y_test):
        self.self = self
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def preprocessing(self):
        self['time:timestamp'] = pd.to_datetime(self['time:timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
        self['time:timestamp'] = (self['time:timestamp'] - self['time:timestamp'].min()) / np.timedelta64(1,'M')
        
        return self
    
    def column_uniquify(self):
        cols = pd.Series(self.columns)
        dup_count = cols.value_counts()
        for dup in cols[cols.duplicated()].unique():
            cols[cols[cols == dup].index.values.tolist()] = [dup + str(i) for i in range(1, dup_count[dup]+1)]
        self.columns = cols
        return self.columns

    def data_split(self):
        X = df.drop('label', axis=1)
        y = df['label']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023, shuffle=True)
        
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        return X_train, y_train, X_test, y_test        
    
    def imbalance(self, X_train, y_train):
        sm = SMOTE(random_state=1)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
        return X_train_res, y_train_res
    
    def DT(self, X_train, y_train, X_test, y_test):
        clf_dt = tree.DecisionTreeClassifier(max_depth=2, random_state=2023, max_features='log2')
        clf_dt = clf_dt.fit(X_train, y_train)
        predictions = clf_dt.predict(X_test)
        accuracy = clf_dt.score(X_test, y_test)
        print('DT Accuracy:', accuracy)
        print('DT AUC Score {}'.format(roc_auc_score(y_test, predictions)))


    def RF(self, X_train, y_train, X_test, y_test):
        clf_rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=2023)
        clf_rf = clf_rf.fit(X_train, y_train)
        predictions = clf_rf.predict(X_test)
        accuracy = clf_rf.score(X_test, y_test)
        print('RF Accuracy:', accuracy)
        print('RF AUC Score {}'.format(roc_auc_score(y_test, predictions)))
    
    def XGB(self, X_train, y_train, X_test, y_test):
        xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=2023)
        xgb.fit(X_train, y_train)
        xgb_pred = xgb.predict(X_test)

        accuracy = accuracy_score(y_test, xgb_pred)
        precision = precision_score(y_test, xgb_pred)
        recall = recall_score(y_test, xgb_pred)
        f1 = f1_score(y_test, xgb_pred)
        
        print('XGB Accuracy:', accuracy)
        print('XGB AUC Score {}'.format(roc_auc_score(y_test, xgb_pred)))

    def LGBM(self, X_train, y_train, X_test, y_test):
        clf_lgbm = LGBMClassifier(random_state=2023)
        clf_lgbm = clf_lgbm.fit(X_train, y_train)
        predictions = clf_lgbm.predict(X_test)
        accuracy = clf_lgbm.score(X_test, y_test)
        print('LGBM Accuracy:', accuracy)
        print('LGBM AUC Score {}'.format(roc_auc_score(y_test, predictions)))

In [4]:
df = AccuracyScore.preprocessing(df)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.columns = AccuracyScore.column_uniquify(df)
X_train, y_train, X_test, y_test = AccuracyScore.data_split(df)
# X_train, y_train = AccuracyScore.imbalance(df, X_train, y_train)

In [5]:
AccuracyScore.DT(df, X_train, y_train, X_test, y_test)
AccuracyScore.RF(df, X_train, y_train, X_test, y_test)
AccuracyScore.XGB(df, X_train, y_train, X_test, y_test)
AccuracyScore.LGBM(df, X_train, y_train, X_test, y_test)

DT Accuracy: 0.789712556732224
DT AUC Score 0.5
RF Accuracy: 0.789712556732224
RF AUC Score 0.5
XGB Accuracy: 0.8328290468986385
XGB AUC Score 0.6143946911436369
LGBM Accuracy: 0.8018154311649016
LGBM AUC Score 0.6158728189861904


In [15]:
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0  709]
 [   1 2373]]


In [16]:
y_test.value_counts()

1    1044
0     278
Name: label, dtype: int64